# #5 Nanogram Game - Formulation
_Author: Éder Pinheiro_  
_Aug, 2021_

This is the MIP formulation of the puzzle. Statement and solution implementation of all puzzles 
are available from the main page of the [Fun Puzzles](https://mip-master.github.io/puzzles/) project, 
which is maintained by [Mip Master](https://mipmaster.org/).

The purpose of the game is to darken some of the cells so that in every row and every collumn the darkened cells form distinct strings with the lengths in the correct order prescribed by the numbers on the left of the row and on top of the column.


### Input Data  
Parameters:

$n,m$ numbers of rows and columns, respectivaly.

$S_c[j]$ list of string sizes in each collumn $j$.

$S_r[i]$ list of string sizes in each row $i$.

### Decision Variables
$x_{ijs}$ if string s of column j starts in row i

$y_{ijs}$ if string s of row i starts in column j

$z_{ij}$ if cell with row $i$ an column $j$ is darken

### Constraints

* Every strings in solution.
$$\sum_{i = 0}^{n-S_c[j][s]} x_{ijs} = 1, \quad j = 0, \cdots, m-1,  s = 0, \cdots, |S_c[j]|-1.$$
$$\sum_{j = 0}^{m-S_r[i][s]} y_{ijs} = 1, \quad   i = 0, \cdots, n-1,  s = 0 \cdots, |S_r[i]|-1.$$

* Strings sequence in right order.
$$x_{ijs} \leq \sum_{k=i+S_c[j][s]}^{n-S_c[j][s+1]+1} x_{kj(s+1)}, \quad  j = 0, \cdots, m-1,  s = 0, \cdots, (|S_c[j]|-2).$$
$$y_{ijs} \leq \sum_{k=j+S_r[i][s]}^{m-S_r[i][s+1]+1} y_{ik(s+1)}, \quad  i = 0, \cdots, n-1,  s = 0, \cdots, (|S_r[i]|-2).$$

* The cells where the strings are placed are daken.
$$z_{ij} \geq  x_{kjs}; \quad  j = 0, \cdots, m-1; \quad s = 0, \cdots, |S_c[j]|-1; \quad k = i-S_c[j][s], \cdots, i.$$.
$$z_{ij} \geq  y_{iks}; \quad i = 0, \cdots, n-1, \quad s = 0, \cdots, |S_r[i]|-1; \quad k = j-S_r[i][s], \cdots, j.$$

* The cells where the strings are not placed must be empty.
$$z_{ij} \leq \sum_{s=0}^{|S_c[j]|-1} \sum_{k=i-S_c[j][s]}^{i} x_{kjs}.$$
$$z_{ij} \leq \sum_{s=0}^{|S_r[i]|-1} \sum_{k=j-S_r[i][s]}^{j} y_{iks}.$$

### Objective Function
$$\min \sum_{i=0}^m\sum_{j=0}^n z_{ij}.$$

## Implementation

In [96]:
import pulp
Sc=[[1,1,3],[1,1,1],[1,1,1],[1,2,1],[3,3],[3,2,3],[3,4],[1,1],[1,2,1,2],[6,2]]
Sr=[[4,1,2],[3,1],[1,3,2],[1,1,1,2],[1,1,1,1],[1,1,1,1],[1,1,1,4],[1,3],[1,4,2],[2,2]]
m = len(Sc)
n = len(Sr)
keys_x=[(i,j,s) for j in range(m) for s in range(len(Sc[j])) for i in range(n)]
keys_y=[(i,j,s) for i in range(n) for s in range(len(Sr[i])) for j in range(m)]
x = pulp.LpVariable.dicts(indexs=keys_x, cat=pulp.LpBinary, name='x')
y = pulp.LpVariable.dicts(indexs=keys_y, cat=pulp.LpBinary, name='y')
z = pulp.LpVariable.dicts(indexs=[(i,j) for i in range(n) for j in range(m)], cat=pulp.LpBinary, name='z')
model = pulp.LpProblem('', sense=pulp.LpMinimize)

for j in range(m):
    for s in range(len(Sc[j])):
        model.addConstraint(pulp.lpSum(x[i, j, s] for i in range(n-Sc[j][s]+1)) == 1, name=f'column_string_{s}{j}')
for i in range(n):
    for s in range(len(Sr[i])):
        model.addConstraint(pulp.lpSum(y[i, j, s] for j in range(m-Sr[i][s]+1)) == 1, name=f'row_string_{s}{i}')
        
for j in range(m):
    for s in range(len(Sc[j])-1):
        for i in range(n):
            model.addConstraint(x[i,j,s] <= pulp.lpSum(x[k, j, s+1] for k in range(i+Sc[j][s]+1,n-Sc[j][s+1]+1)), name=f'{Sc[j][s]}_string_{s}column{j}_above_{s+1}{j}_row_{i}')      
for i in range(n):
    for s in range(len(Sr[i])-1):
        for j in range(m):
            model.addConstraint(y[i,j,s] <= pulp.lpSum(y[i, k, s+1] for k in range(j+Sr[i][s]+1,m-Sr[i][s+1]+1)), name=f'{Sr[i][s]}_string_{s}row{i}_left_from_{s+1}{i}_col_{j}')
        
for i in range(n):
    for j in range(m):
        for s in range(len(Sc[j])):
            for k in range(max(i-Sc[j][s]+1,0),i+1):
                model.addConstraint(z[i,j] >= x[k, j, s], name=f'z_{i}{j}_x_{k}{j}{s}')
for j in range(m):
    for i in range(n):
        for s in range(len(Sr[i])):
            for k in range(max(j-Sr[i][s]+1,0), j+1):
                model.addConstraint(z[i,j] >= y[i, k, s], name=f'z_{i}{j}_y_{i}{k}{s}')
            
for i in range(n):
    for j in range(m):
        model.addConstraint(z[i,j] <= pulp.lpSum(x[k, j, s] for s in range(len(Sc[j])) for k in range(max(i-Sc[j][s]+1,0),i+1)), name=f'z_{i}{j}_sumx')
for i in range(n):
    for j in range(m):
        model.addConstraint(z[i,j] <= pulp.lpSum(y[i, k, s] for s in range(len(Sr[i])) for k in range(max(j-Sr[i][s]+1,0),j+1)), name=f'z_{i}{j}_sumy')
        
        
# set the objective function
model.setObjective(pulp.lpSum(z[(i, j)] for i in range(n) for j in range(m)))


# optimize
model.solve()

# retrieve and print out the solution
z_sol = [[z[i,j].value() for j in range(m)] for i in range(n)]
# print(f'z = {z_sol}')
for row in z_sol:
    print(row)